MDA FinalProject 106062132 趙仰生

In [1]:
from pyspark import SparkConf , SparkContext

ˇ第一個mapper用來存取資料，key為movie ID，value為user ID及rating

In [2]:
def mapper1(line):
    wordlist = line.split("\t")
    maplist = []
    maplist.append((int(wordlist[1]),[int(wordlist[0]),float(wordlist[2])]))
    return maplist

第一個reducer直接相加即可。

In [3]:
def reducer1(x,y):
    return x+y

將data經過mapper1及reducer1的處理。

In [4]:
conf = SparkConf().setMaster("local[*]") \
    .setAppName("App_Name") \
    .set('spark.executor.memory', '4G') \
    .set('spark.driver.memory', '45G') \
    .set('spark.driver.maxResultSize', '10G')
sc = SparkContext(conf=conf)
lines = sc.textFile("data2.txt").flatMap(mapper1).reduceByKey(reducer1)

第二個mapper用來計算cosine sim的分母(所有user對某個movie的rating平方相加開根號)，key為movie ID，value為其分母之值。

In [5]:
def mapper2(line):
    maplist = []
    i = 0
    j = 1
    sum = 0
    while 1:
        if(i*2 == len(line[1])):
            break
        if(line[1][i * 2] == j):
            sum += (line[1][i*2 + 1])**2
            i += 1
        j += 1
    maplist.append((line[0],sum**0.5))
    return maplist

執行mapper2

In [6]:
lines = lines.flatMap(mapper2)

利用陣列rating_array，存取所有movie各自的分母。

In [7]:
lines = lines.collect()
rating_array = [0] * 193610
for item in lines:
    rating_array[item[0]] = item[1]

第三個mapper用來再次處理資料，目的是為了待會方便計算cosine sim的分子。其key為user ID，value則為movie ID及rating。

In [8]:
def mapper3(line):
    wordlist = line.split("\t")
    maplist = []
    maplist.append((int(wordlist[0]),[int(wordlist[1]),float(wordlist[2])]))
    return maplist

利用mapper3及reducer1處理資料。

In [9]:
lines2 = sc.textFile("data2.txt").flatMap(mapper3).reduceByKey(reducer1)

第四個mapper用來計算分子各項/分母，key為movie ID的pair，value則為cosine similarity的分子各項/分母(分母利用剛剛存起來的rating_array)。

In [10]:
def mapper4(line):
    maplist = []
    for i in range(len(line[1])):
        for j in range(len(line[1])):
            if(j > i and i%2 == 0 and j%2 == 0):
                maplist.append(((line[1][i] , line[1][j]), round(line[1][i+1] * line[1][j+1] / rating_array[line[1][i]] / rating_array[line[1][j]] , 5)))
    return maplist

第五個mapper用來交換key跟value的位置，為了方便排序similarity。

In [11]:
def mapper5(line):
    maplist = []
    maplist.append((line[1],line[0]))
    return maplist

將資料經過mapper4、reducer1、mapper5的運算，並且sortbykey，由大到小將similarity排序。

In [12]:
lines2 = lines2.flatMap(mapper4).reduceByKey(reducer1)
lines2 = lines2.flatMap(mapper5).sortByKey(ascending= False)

輸出output

In [13]:
f = open('Outputfile2.txt', 'w')
f.write(str(lines2.collect()))
f.close()

In [14]:
sc.stop()